In [1]:
# ************************************Import lib for workflow
import pandas as pd
import numpy as np
import os
#import json
#import calendar
#import glob
#import re
# pyproj and utm for Lat Long conversion
#from pyproj import Proj
import utm

In [9]:


#***********************************************FUNCTIONS:

#******************************************Deviation survey functions
# gets UTM coords from lat long
def getUTMs(row):
    """
    DESCRIPTION
    """
    
    tup = utm.from_latlon(row.iloc[0],row.iloc[1])
    return pd.Series(tup[:4])

#gets lat long from UTM coords
def getLatLon(row):
    """
    DESCRIPTION
    """
    
    tup = utm.to_latlon(row.iloc[0],row.iloc[1],row.iloc[2],row.iloc[3])
    return pd.Series(tup[:2])

def calcXYOffset(dfDirSrvy):
    """
    DESCRIPTION
    """
    
    #X_OFFSET is equal to E/W deviation when E is positive and W is negative
    #Y_OFFSET is equal to N/S deviation when N is positive and S is negative
    
    # create dict to map for offset
    offsetDict = {
        "E" : 1,
        "W" : -1,
        "N" : 1,
        "S" : -1
    }
    # create new columns and map dict * the deviations
    dfDirSrvy['X_Offset']= dfDirSrvy['E/W'].map(offsetDict)*dfDirSrvy['Deviation E/W']
    dfDirSrvy['Y_Offset']= dfDirSrvy['N/S'].map(offsetDict)*dfDirSrvy['Deviation N/S']

    #reset index
    dfDirSrvy = dfDirSrvy.reset_index(drop=True)
    return dfDirSrvy



#***************************************************************************

def createDirSrvy():
    """
    IHS Dev Survey
    Bring in Directional Surveys and preprocessed well header
    create X and Y Offset columns 
    Bring in Surface lat and lon for each well 
    Convert Surface lat and lon to X and Y 
    Add the X and Y Offset to the new Surface X and Y to create adjusted X and Ys 
    Convert the adjusted X and Ys to Lat Lon
    
    May take a while depending on data set size
    """
    
    
    #**************************************
    # PROCESSED WELL HEADER IS REQUIRED
    # find and bring in processed well header
    fileNm = os.path.join(
        DATA_PATH, "Processed", "IHSWellHeader.csv")
    
    datacols = ['UWI','Surface Latitude','Surface Longitude']
    
    # read csv, dropna if present, convert uwi to string
    df = pd.read_csv(fileNm, sep=',',usecols=datacols)
    df = df.dropna()
    df['UWI'] = df['UWI'].apply(str)

    print(f'well header length: {len(df)}.')
    print(f'well header length Drop Duplicates: {len(df.drop_duplicates())}.')
    #**********************************************************
    
    # Grab files and append multiple dev survey files together if available
    dfDirSrvy = appendDfs(DATA_PATH,'xlsx',sheetName='Survey')

    #convert UWI to string
    dfDirSrvy['UWI'] = dfDirSrvy['UWI'].apply(str)

    # calculate the xyOffsets
    dfDirSrvy = calcXYOffset(dfDirSrvy)

    # convert surface x and y to lat and long then add the X and Y offsets from the surveys
    # then convert the adjusted x and y's to lat and long.

    # convert surface x and y to surface lat and long
    df[['Surface_X','Surface_Y','ZoneNumber','ZoneLetter']] = df[['Surface Latitude','Surface Longitude']].apply(getUTMs , axis=1)
    print('converted surface x and y to surface lat and long')

    #merge the new surface x and y and surface lat lon with the deviation dataframe for each uwi
    df = pd.merge(dfDirSrvy, df, on='UWI', how='left')

    # create X and Y columns for each deviation point per uwi
    # add the x and y offset to the surface x and y for each uwi
    df['X_Adjusted'] = df['Surface_X']+(df['X_Offset']*0.3048)
    df['Y_Adjusted'] = df['Surface_Y']+(df['Y_Offset']*0.3048)

    # convert adjusted x and y back to lat long
    df[['Lat_adjusted','Lon_adjusted']] = df[['X_Adjusted','Y_Adjusted','ZoneNumber','ZoneLetter']].apply(getLatLon , axis=1)
    print('converted adjusted x and y back to lat long')

    #drop columns used in lat lon calc
    df.drop(['Y_Adjusted','X_Adjusted','ZoneLetter',
              'ZoneNumber','Surface_Y','Surface_X'], axis=1, inplace=True)


    #export data
    exportPath = DATA_PATH +'Processed/IHSWellDirSrvy.csv'
    df.to_csv(os.path.join(exportPath),index=False)
    print('EXPORTED: IHSWellDirSrvy.csv')
    return df

In [3]:
#define column names:

def minCurveAlgo(df,uwi,wellName,md,inc,azim):
        
        #name cols
        incRad = 'INCLINATION_rad'
        azimRad = 'AZIMUTH_rad'


        #Convert to Radians


        df[incRad] = df[inc]*0.0174533 #converting to radians
        df[azimRad] =df[azim]*0.0174533 #converting to radians

        # ************************************************ BETA CALC 
        df['beta'] = np.arccos(
                     np.cos((df[incRad]) - (df[incRad].shift(1))) - \
                     (np.sin(df[incRad].shift(1)) * np.sin(df[incRad]) * \
                     (1-np.cos(df[azimRad] - df[azimRad].shift(1)))))

        df['beta'] = df['beta'].fillna(0)

        # *************************************************BETA CALC END

        #DogLeg Severity per 100 ft

        df['DLS_Sub'] = (df['beta'] * 57.2958 * 100)/(df[md]-df[md].shift(1))
        df['DLS_Sub'] = df['DLS_Sub'].fillna(0)
        
        # Calc RF
        df['RF'] = np.where(df['beta']==0, 1, 2/df['beta'] * np.tan(df['beta']/2))


        # ************************************************************** TVD CALC

        df['TVD_Sub'] = ((df[md]-df[md].shift(1))/2) * \
                        (np.cos(df[incRad].shift(1)) + np.cos(df[incRad]))*df['RF']

        df['TVD_Sub'] = df['TVD_Sub'].fillna(0)
        df['TVD_SubCUM'] =  df['TVD_Sub'].cumsum()

        ### calculating NS
        df['NS_Sub'] = ((df[md]-df[md].shift(1))/2) * \
                        (
                        np.sin(df[incRad].shift(1)) * np.cos(df[azimRad].shift(1)) +
                        np.sin(df[incRad]) * np.cos(df[azimRad])\
                        ) * df['RF']

        df['NS_Sub'] = df['NS_Sub'].fillna(0)
        df['NS_SubCUM'] =  df['NS_Sub'].cumsum()

        ## calculating EW
        df['EW_Sub'] = ((df[md]-df[md].shift(1))/2) * \
                        (
                        np.sin(df[incRad].shift(1)) * \
                        np.sin(df[azimRad].shift(1)) + \
                        np.sin(df[incRad]) * np.sin(df[azimRad])\
                        ) * df['RF']
        df['EW_Sub'] = df['EW_Sub'].fillna(0)
        df['EW_SubCUM'] =  df['EW_Sub'].cumsum()
        
        return df


def createSurvey(dfWells,uwi,wellName,md,inc,azim):
    
    dfWells.rename(columns={uwi:'UWI',
                            wellName:'WellName',
                                  md:'MD',
                          inc:'INCLINATION',
                          azim:'AZIMUTH'}, inplace=True)

    wellids = dfWells[uwi].unique()
    
    #create empty df for appending
    appended_data = pd.DataFrame()

    #Loop over all wells
    for well in wellids:
        
        #Following are the calculations for Minimum Curvature Method 
        #find df of each uwi
        df = dfWells.loc[dfWells[uwi] == str(well)].reset_index(drop=True)
        
        #run min curve algo
        minCurveAlgo(df,uwi,md,inc,azim)
        
        appended_data = appended_data.append(df)
    
    appended_data.rename(columns={'TVD_SubCUM':'TVD',
                                  'DLS_Sub':'DLS',
                          'NS_SubCUM':'NS_OFFSET',
                          'EW_SubCUM':'EW_OFFSET'}, inplace=True)
    
    appended_data = appended_data[['UWI','WellName','MD','INCLINATION','AZIMUTH','TVD','NS_OFFSET','EW_OFFSET','INCLINATION_rad','AZIMUTH_rad','DLS']]
        
    return appended_data

In [61]:
file = "C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/Well_A_Dir_Survey.csv"

dfwells = pd.read_csv(file, sep=',')
#dfwells = dfwells[['UWI','Measured Depth','Deviation Angle','Deviation Azimuth','Deviation N/S',]]
dfwells.tail()

,UWI,Measured Depth,TV Depth,Deviation Angle,Deviation Azimuth,Deviation N/S,N/S,Deviation E/W,E/W,PRODFit Formation Code,PRODFit Formation Name,Point Type Code,Point Type Name
36,Well_A,9099,8023.49,91.1,226.5,907.19,S,891.66,W,NaN,NaN,NaN,NaN
37,Well_A,9161,8021.59,92.4,226.1,950.00,S,936.47,W,NaN,NaN,NaN,NaN
38,Well_A,9224,8019.34,91.7,226.5,993.50,S,981.98,W,NaN,NaN,NaN,NaN
39,Well_A,9285,8017.05,92.6,226.1,1035.62,S,1026.05,W,NaN,NaN,NaN,NaN
40,Well_A,9328,8015.10,92.6,226.1,1065.40,S,1057.00,W,NaN,NaN,PE,"PROJECTED, END POINT"


In [62]:
file = "C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/well_surface_location.csv"

df_surface = pd.read_csv(file, sep=',')
df_surface.tail()

,UWI,Surface Latitude,Surface Longitude
0,Well_A,33.438389,-97.482414


In [63]:
df = pd.merge(dfwells,df_surface,on="UWI")
df.tail()

,UWI,Measured Depth,TV Depth,Deviation Angle,Deviation Azimuth,Deviation N/S,N/S,Deviation E/W,E/W,PRODFit Formation Code,PRODFit Formation Name,Point Type Code,Point Type Name,Surface Latitude,Surface Longitude
36,Well_A,9099,8023.49,91.1,226.5,907.19,S,891.66,W,NaN,NaN,NaN,NaN,33.438389,-97.482414
37,Well_A,9161,8021.59,92.4,226.1,950.00,S,936.47,W,NaN,NaN,NaN,NaN,33.438389,-97.482414
38,Well_A,9224,8019.34,91.7,226.5,993.50,S,981.98,W,NaN,NaN,NaN,NaN,33.438389,-97.482414
39,Well_A,9285,8017.05,92.6,226.1,1035.62,S,1026.05,W,NaN,NaN,NaN,NaN,33.438389,-97.482414
40,Well_A,9328,8015.10,92.6,226.1,1065.40,S,1057.00,W,NaN,NaN,PE,"PROJECTED, END POINT",33.438389,-97.482414


In [64]:
df = calcXYOffset(df)
df.tail()

,UWI,Measured Depth,TV Depth,Deviation Angle,Deviation Azimuth,Deviation N/S,N/S,Deviation E/W,E/W,PRODFit Formation Code,PRODFit Formation Name,Point Type Code,Point Type Name,Surface Latitude,Surface Longitude,X_Offset,Y_Offset
36,Well_A,9099,8023.49,91.1,226.5,907.19,S,891.66,W,NaN,NaN,NaN,NaN,33.438389,-97.482414,-891.66,-907.19
37,Well_A,9161,8021.59,92.4,226.1,950.00,S,936.47,W,NaN,NaN,NaN,NaN,33.438389,-97.482414,-936.47,-950.00
38,Well_A,9224,8019.34,91.7,226.5,993.50,S,981.98,W,NaN,NaN,NaN,NaN,33.438389,-97.482414,-981.98,-993.50
39,Well_A,9285,8017.05,92.6,226.1,1035.62,S,1026.05,W,NaN,NaN,NaN,NaN,33.438389,-97.482414,-1026.05,-1035.62
40,Well_A,9328,8015.10,92.6,226.1,1065.40,S,1057.00,W,NaN,NaN,PE,"PROJECTED, END POINT",33.438389,-97.482414,-1057.00,-1065.40


In [65]:
SurfaceLatitude = 'Surface Latitude'
SurfaceLongitude ='Surface Longitude'
X_offset = 'X_Offset'
Y_offset = 'Y_Offset'

# drop rows that dont have surface locs
df = df.dropna(subset=[SurfaceLatitude])
df = df.dropna(subset=[X_offset])
# calculate the xyOffsets (if offsets are all postive values, this will make them neg and pos in the correct place)
#dfDirSrvy = calcXYOffset(dfDirSrvy)

# convert surface x and y to lat and long then add the X and Y offsets from the surveys
# then convert the adjusted x and y's to lat and long.

# convert surface x and y to surface lat and long
df[['Surface_X','Surface_Y','ZoneNumber','ZoneLetter']] = df[[SurfaceLatitude,SurfaceLongitude]].apply(getUTMs , axis=1)
print('converted surface x and y to surface lat and long')


# create X and Y columns for each deviation point per uwi
# add the x and y offset to the surface x and y for each uwi
df['X_Adjusted'] = df['Surface_X']+(df[X_offset]*0.3048)
df['Y_Adjusted'] = df['Surface_Y']+(df[Y_offset]*0.3048)

# convert adjusted x and y back to lat long
df[['Lat_adjusted','Lon_adjusted']] = df[['X_Adjusted','Y_Adjusted','ZoneNumber','ZoneLetter']].apply(getLatLon , axis=1)
print('converted adjusted x and y back to lat long')

#drop columns used in lat lon calc
#df.drop(['ZoneLetter',
#          'ZoneNumber','Surface_Y','Surface_X'], axis=1, inplace=True)


#export data
#exportPath = DATA_PATH +'Processed/IHSWellDirSrvy.csv'
#df.to_csv(os.path.join(exportPath),index=False)
#print('EXPORTED: IHSWellDirSrvy.csv')

converted surface x and y to surface lat and long
converted adjusted x and y back to lat long


In [66]:
df.tail(10)

,UWI,Measured Depth,TV Depth,Deviation Angle,Deviation Azimuth,Deviation N/S,N/S,Deviation E/W,E/W,PRODFit Formation Code,...,X_Offset,Y_Offset,Surface_X,Surface_Y,ZoneNumber,ZoneLetter,X_Adjusted,Y_Adjusted,Lat_adjusted,Lon_adjusted
31,Well_A,8783,8031.00,92.7,228.3,693.71,S,658.84,W,NaN,...,-658.84,-693.71,641067.861785,3.700918e+06,14,S,640867.047353,3.700707e+06,33.436509,-97.484606
32,Well_A,8847,8028.49,91.8,228.3,736.26,S,706.58,W,NaN,...,-706.58,-736.26,641067.861785,3.700918e+06,14,S,640852.496201,3.700694e+06,33.436394,-97.484765
33,Well_A,8910,8026.90,91.1,226.9,778.72,S,753.09,W,NaN,...,-753.09,-778.72,641067.861785,3.700918e+06,14,S,640838.319953,3.700681e+06,33.436279,-97.484919
34,Well_A,8973,8026.13,90.3,227.6,821.48,S,799.35,W,NaN,...,-799.35,-821.48,641067.861785,3.700918e+06,14,S,640824.219905,3.700668e+06,33.436163,-97.485073
35,Well_A,9036,8025.03,91.7,227.2,864.12,S,845.72,W,NaN,...,-845.72,-864.12,641067.861785,3.700918e+06,14,S,640810.086329,3.700655e+06,33.436048,-97.485227
36,Well_A,9099,8023.49,91.1,226.5,907.19,S,891.66,W,NaN,...,-891.66,-907.19,641067.861785,3.700918e+06,14,S,640796.083817,3.700642e+06,33.435931,-97.485380
37,Well_A,9161,8021.59,92.4,226.1,950.00,S,936.47,W,NaN,...,-936.47,-950.00,641067.861785,3.700918e+06,14,S,640782.425729,3.700629e+06,33.435816,-97.485529
38,Well_A,9224,8019.34,91.7,226.5,993.50,S,981.98,W,NaN,...,-981.98,-993.50,641067.861785,3.700918e+06,14,S,640768.554281,3.700615e+06,33.435698,-97.485680
39,Well_A,9285,8017.05,92.6,226.1,1035.62,S,1026.05,W,NaN,...,-1026.05,-1035.62,641067.861785,3.700918e+06,14,S,640755.121745,3.700602e+06,33.435584,-97.485826
40,Well_A,9328,8015.10,92.6,226.1,1065.40,S,1057.00,W,NaN,...,-1057.00,-1065.40,641067.861785,3.700918e+06,14,S,640745.688185,3.700593e+06,33.435503,-97.485929


# Convert MD, INCL, AZIM to TVD, X and Y offset

In [67]:
df.columns

Index(['UWI', 'Measured Depth', 'TV Depth', 'Deviation Angle',
       'Deviation Azimuth', 'Deviation N/S', 'N/S', 'Deviation E/W', 'E/W',
       'PRODFit Formation Code', 'PRODFit Formation Name', 'Point Type Code',
       'Point Type Name', 'Surface Latitude', 'Surface Longitude', 'X_Offset',
       'Y_Offset', 'Surface_X', 'Surface_Y', 'ZoneNumber', 'ZoneLetter',
       'X_Adjusted', 'Y_Adjusted', 'Lat_adjusted', 'Lon_adjusted'],
      dtype='object')

In [68]:
df_sub = df[['UWI','Measured Depth','Deviation Angle','Deviation Azimuth']]
df_sub.rename(columns={'Measured Depth':'MD','Deviation Angle':'INC','Deviation Azimuth':'AZIM'},inplace=True)
df_sub.head()

C:\Users\BpAmos\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,UWI,MD,INC,AZIM
0,Well_A,7654,2.0,256.3
1,Well_A,7700,1.9,253.3
2,Well_A,7743,6.5,227.9
3,Well_A,7774,12.1,238.8
4,Well_A,7806,17.8,234.9


In [70]:
#define column names:
uwi = 'UWI'
md = 'MD'
inc = 'INC'
azim = 'AZIM'
incRad = 'INCLINATION_rad'
azimRad = 'AZIMUTH_rad'



wellids = df_sub[uwi].unique()
appended_data = pd.DataFrame()

#Loop over all wells
for well in wellids:
    #Following are the calculations for Minimum Curvature Method 
    #find df of each uwi
    
    df = df_sub.loc[df_sub[uwi] == str(well)].reset_index()


    #Convert to Radians
    
    
    df[incRad] = df[inc]*0.0174533 #converting to radians
    df[azimRad] =df[azim]*0.0174533 #converting to radians
    
    # ************************************************ BETA CALC 
    df['beta'] = np.arccos(
                 np.cos((df[incRad]) - (df[incRad].shift(1))) - \
                 (np.sin(df[incRad].shift(1)) * np.sin(df[incRad]) * \
                 (1-np.cos(df[azimRad] - df[azimRad].shift(1)))))

    df['beta'] = df['beta'].fillna(0)

    # *************************************************BETA CALC END
    
    #DogLeg Severity per 100 ft
    
    df['DLS_Sub'] = (df['beta'] * 57.2958 * 100)/(df[md]-df[md].shift(1))
    
    # Calc RF
    df['RF'] = np.where(df['beta']==0, 1, 2/df['beta'] * np.tan(df['beta']/2))
    
    
    # ************************************************************** TVD CALC

    df['TVD_Sub'] = ((df[md]-df[md].shift(1))/2) * \
                    (np.cos(df[incRad].shift(1)) + np.cos(df[incRad]))*df['RF']
    
    df['TVD_Sub'] = df['TVD_Sub'].fillna(0)
    df['TVD_SubCUM'] =  df['TVD_Sub'].cumsum()
    
    ### calculating NS
    df['NS_Sub'] = ((df[md]-df[md].shift(1))/2) * \
                    (
                    np.sin(df[incRad].shift(1)) * np.cos(df[azimRad].shift(1)) +
                    np.sin(df[incRad]) * np.cos(df[azimRad])\
                    ) * df['RF']
    
    df['NS_Sub'] = df['NS_Sub'].fillna(0)
    df['NS_SubCUM'] =  df['NS_Sub'].cumsum()
    
    ## calculating EW
    df['EW_Sub'] = ((df[md]-df[md].shift(1))/2) * \
                    (
                    np.sin(df[incRad].shift(1)) * \
                    np.sin(df[azimRad].shift(1)) + \
                    np.sin(df[incRad]) * np.sin(df[azimRad])\
                    ) * df['RF']
    df['EW_Sub'] = df['EW_Sub'].fillna(0)
    df['EW_SubCUM'] =  df['EW_Sub'].cumsum()
    
    
    appended_data = appended_data.append(df)

In [72]:
appended_data.tail()

,index,UWI,MD,INC,AZIM,INCLINATION_rad,AZIMUTH_rad,beta,DLS_Sub,RF,TVD_Sub,TVD_SubCUM,NS_Sub,NS_SubCUM,EW_Sub,EW_SubCUM
36,36,Well_A,9099,91.1,226.5,1.589996,3.953172,0.016088,1.463165,1.000022,-1.539282,372.077441,-43.073005,-814.705290,-45.948463,-891.224679
37,37,Well_A,9161,92.4,226.1,1.612685,3.946191,0.023738,2.193695,1.000047,-1.893397,370.184044,-42.813595,-857.518884,-44.802128,-936.026807
38,38,Well_A,9224,91.7,226.5,1.600468,3.953172,0.014069,1.279521,1.000016,-2.253650,367.930394,-43.497263,-901.016147,-45.517510,-981.544316
39,39,Well_A,9285,92.6,226.1,1.616176,3.946191,0.017187,1.614378,1.000025,-2.288490,365.641904,-42.113520,-943.129667,-44.069521,-1025.613837
40,40,Well_A,9328,92.6,226.1,1.616176,3.946191,0.000000,0.000000,1.000000,-1.950638,363.691265,-29.785532,-972.915199,-30.951852,-1056.565689


In [75]:
df = pd.merge(appended_data,df_surface,on="UWI")
df.tail()

,index,UWI,MD,INC,AZIM,INCLINATION_rad,AZIMUTH_rad,beta,DLS_Sub,RF,TVD_Sub,TVD_SubCUM,NS_Sub,NS_SubCUM,EW_Sub,EW_SubCUM,Surface Latitude,Surface Longitude
36,36,Well_A,9099,91.1,226.5,1.589996,3.953172,0.016088,1.463165,1.000022,-1.539282,372.077441,-43.073005,-814.705290,-45.948463,-891.224679,33.438389,-97.482414
37,37,Well_A,9161,92.4,226.1,1.612685,3.946191,0.023738,2.193695,1.000047,-1.893397,370.184044,-42.813595,-857.518884,-44.802128,-936.026807,33.438389,-97.482414
38,38,Well_A,9224,91.7,226.5,1.600468,3.953172,0.014069,1.279521,1.000016,-2.253650,367.930394,-43.497263,-901.016147,-45.517510,-981.544316,33.438389,-97.482414
39,39,Well_A,9285,92.6,226.1,1.616176,3.946191,0.017187,1.614378,1.000025,-2.288490,365.641904,-42.113520,-943.129667,-44.069521,-1025.613837,33.438389,-97.482414
40,40,Well_A,9328,92.6,226.1,1.616176,3.946191,0.000000,0.000000,1.000000,-1.950638,363.691265,-29.785532,-972.915199,-30.951852,-1056.565689,33.438389,-97.482414


In [78]:
SurfaceLatitude = 'Surface Latitude'
SurfaceLongitude ='Surface Longitude'
X_offset = 'NS_Sub'
Y_offset = 'EW_Sub'

# drop rows that dont have surface locs
df = df.dropna(subset=[SurfaceLatitude])
df = df.dropna(subset=[X_offset])
# calculate the xyOffsets (if offsets are all postive values, this will make them neg and pos in the correct place)
#dfDirSrvy = calcXYOffset(dfDirSrvy)

# convert surface x and y to lat and long then add the X and Y offsets from the surveys
# then convert the adjusted x and y's to lat and long.

# convert surface x and y to surface lat and long
df[['Surface_X','Surface_Y','ZoneNumber','ZoneLetter']] = df[[SurfaceLatitude,SurfaceLongitude]].apply(getUTMs , axis=1)
print('converted surface x and y to surface lat and long')


# create X and Y columns for each deviation point per uwi
# add the x and y offset to the surface x and y for each uwi
df['X_Adjusted'] = df['Surface_X']+(df[X_offset]*0.3048)
df['Y_Adjusted'] = df['Surface_Y']+(df[Y_offset]*0.3048)

# convert adjusted x and y back to lat long
df[['Lat_adjusted','Lon_adjusted']] = df[['X_Adjusted','Y_Adjusted','ZoneNumber','ZoneLetter']].apply(getLatLon , axis=1)
print('converted adjusted x and y back to lat long')

#drop columns used in lat lon calc
#df.drop(['ZoneLetter',
#          'ZoneNumber','Surface_Y','Surface_X'], axis=1, inplace=True)


#export data
#exportPath = DATA_PATH +'Processed/IHSWellDirSrvy.csv'
#df.to_csv(os.path.join(exportPath),index=False)
#print('EXPORTED: IHSWellDirSrvy.csv')

converted surface x and y to surface lat and long
converted adjusted x and y back to lat long


In [79]:
df

,index,UWI,MD,INC,AZIM,INCLINATION_rad,AZIMUTH_rad,beta,DLS_Sub,RF,...,Surface Latitude,Surface Longitude,Surface_X,Surface_Y,ZoneNumber,ZoneLetter,X_Adjusted,Y_Adjusted,Lat_adjusted,Lon_adjusted
0,0,Well_A,7654,2.0,256.3,0.034907,4.473281,0.000000,NaN,1.000000,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641067.861785,3.700918e+06,33.438389,-97.482413
1,1,Well_A,7700,1.9,253.3,0.033161,4.420921,0.002494,0.310585,1.000001,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641067.401458,3.700918e+06,33.438388,-97.482418
2,2,Well_A,7743,6.5,227.9,0.113446,3.977607,0.084689,11.284417,1.000598,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641067.102792,3.700918e+06,33.438384,-97.482422
3,3,Well_A,7774,12.1,238.8,0.211185,4.167848,0.102032,18.858018,1.000868,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641066.616796,3.700917e+06,33.438381,-97.482427
4,4,Well_A,7806,17.8,234.9,0.310669,4.099780,0.100967,18.078053,1.000850,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641065.765880,3.700917e+06,33.438377,-97.482436
5,5,Well_A,7838,23.6,231.8,0.411898,4.045675,0.102986,18.439601,1.000885,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641065.105310,3.700916e+06,33.438371,-97.482443
6,6,Well_A,7870,29.6,230.4,0.516618,4.021240,0.105283,18.850859,1.000925,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641064.468264,3.700915e+06,33.438365,-97.482450
7,7,Well_A,7901,35.9,228.6,0.626573,3.989824,0.111251,20.561908,1.001033,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641063.981721,3.700915e+06,33.438359,-97.482456
8,8,Well_A,7932,42.0,226.1,0.733039,3.946191,0.109923,20.316618,1.001008,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641063.501549,3.700914e+06,33.438353,-97.482461
9,9,Well_A,7963,46.8,225.1,0.816814,3.928738,0.084659,15.647116,1.000598,...,33.438389,-97.482414,641067.861785,3.700918e+06,14,S,641063.141642,3.700913e+06,33.438348,-97.482465
